In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
tqdm.pandas()

# ✅ 시작
print("🚀 데이터 로드 중...")
df = pd.read_excel(r"C:\Users\user\Desktop\파프\기사_전체_통합본.xlsx")
df["기사수"] = pd.to_numeric(df["기사수"], errors="coerce").fillna(0).astype(int)

X_news_count = df[["기사수"]].values  # shape (n, 1)

print("🔍 결측 제거 및 열 추출 중...")
category_cols = ['통합 분류1', '통합 분류2', '통합 분류3',
                 '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '감성']
df = df.dropna(subset=["설명력", "특성추출(가중치순 상위 50개)"])
y = df["설명력"].astype(int)

# ✅ SBERT 모델 로드
print("🧠 SBERT 임베딩 모델 로딩 중...")
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# ✅ 임베딩 함수
def embed_features(df):
    print("🔡 텍스트 임베딩 중 (SBERT)...")
    texts = df["특성추출(가중치순 상위 50개)"].fillna("").tolist()
    embeddings = []
    for text in tqdm(texts, desc="💬 임베딩 진행"):
        emb = model.encode(text, convert_to_numpy=True)
        embeddings.append(emb)
    return np.array(embeddings)

# ✅ 문장 임베딩
X_embed = embed_features(df)

# ✅ One-hot 인코딩 처리
print("🔠 분류 및 감성 One-hot 인코딩 중...")
onehot = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_cat = onehot.fit_transform(df[category_cols])
print(f"  ➤ 인코딩 결과 shape: {X_cat.shape}")

# ✅ 전체 feature 합치기
print("🔗 임베딩 + 인코딩 합치기...")
X_all = np.concatenate([X_embed, X_cat, X_news_count], axis=1)

print(f"  ➤ 최종 feature shape: {X_all.shape}")

# ✅ 학습-검증 분할
print("🧪 학습/검증 세트 분할 중...")
X_train, X_test, y_train, y_test = train_test_split(X_all, y, test_size=0.2, random_state=42)
print(f"  ➤ 학습 세트: {X_train.shape}, 검증 세트: {X_test.shape}")

# ✅ 모델 학습
print("🎯 랜덤포레스트 분류기 학습 중...")
clf = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced')
clf.fit(X_train, y_train)
print("✅ 학습 완료!")

# ✅ 모델 평가
print("📊 모델 성능 평가:")
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


🚀 데이터 로드 중...
🔍 결측 제거 및 열 추출 중...
🧠 SBERT 임베딩 모델 로딩 중...
🔡 텍스트 임베딩 중 (SBERT)...


💬 임베딩 진행:   8%|█▋                   | 10/123 [02:47<31:30, 16.73s/it]


KeyboardInterrupt: 

In [8]:
def embed_features(df):
    print("🔡 텍스트 임베딩 중 (SBERT)...")
    texts = df["특성추출(가중치순 상위 50개)"].fillna("").tolist()
    embeddings = []
    for text in tqdm(texts, desc="💬 임베딩 진행"):
        emb = model.encode(text, convert_to_numpy=True)
        embeddings.append(emb)
    return np.array(embeddings)


# ✅ 문장 임베딩
X_embed = embed_features(df)

🔡 텍스트 임베딩 중 (SBERT)...


💬 임베딩 진행: 100%|████████████████████| 123/123 [35:27<00:00, 17.29s/it]


In [20]:
import pickle
import pickle

with open(r"C:\Users\user\Desktop\파프\X_embed.pkl", "wb") as f:
    pickle.dump(X_embed, f)  # 가장 보존력 좋음
pd.DataFrame(X_embed).to_csv(r"C:\Users\user\Desktop\파프\X_embed.csv", index=False)# 호환성 좋음

In [40]:
print("🚀 데이터 로드 중...")
df = pd.read_excel(r"C:\Users\user\Desktop\파프\기사_전체_통합본.xlsx")
df["기사수"] = pd.to_numeric(df["기사수"], errors="coerce").fillna(0).astype(int)

X_news_count = df[["기사수"]].values  # shape (n, 1)

print("🔍 결측 제거 및 열 추출 중...")
category_cols = ['통합 분류1', '통합 분류2', '통합 분류3',
                 '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '감성']
df = df.dropna(subset=["설명력", "특성추출(가중치순 상위 50개)"])
y = df["설명력"].astype(int)
# ✅ One-hot 인코딩 처리
print("🔠 분류 및 감성 One-hot 인코딩 중...")
onehot = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_cat = onehot.fit_transform(df[category_cols])
print(f"  ➤ 인코딩 결과 shape: {X_cat.shape}")

# ✅ 전체 feature 합치기
print("🔗 임베딩 + 인코딩 합치기...")
X_all = np.concatenate([X_embed, X_cat, X_news_count], axis=1)

print(f"  ➤ 최종 feature shape: {X_all.shape}")

# ✅ 학습-검증 분할
print("🧪 학습/검증 세트 분할 중...")
X_train, X_test, y_train, y_test = train_test_split(X_all, y, test_size=0.2, random_state=42)
print(f"  ➤ 학습 세트: {X_train.shape}, 검증 세트: {X_test.shape}")

# ✅ 모델 학습
print("🎯 랜덤포레스트 분류기 학습 중...")
clf = RandomForestClassifier(n_estimators=600, random_state=42, max_depth = 10, class_weight='balanced')
clf.fit(X_train, y_train)
print("✅ 학습 완료!")

# ✅ 모델 평가
print("📊 모델 성능 평가:")
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


🚀 데이터 로드 중...
🔍 결측 제거 및 열 추출 중...
🔠 분류 및 감성 One-hot 인코딩 중...
  ➤ 인코딩 결과 shape: (123, 146)
🔗 임베딩 + 인코딩 합치기...
  ➤ 최종 feature shape: (123, 915)
🧪 학습/검증 세트 분할 중...
  ➤ 학습 세트: (98, 915), 검증 세트: (25, 915)
🎯 랜덤포레스트 분류기 학습 중...
✅ 학습 완료!
📊 모델 성능 평가:
              precision    recall  f1-score   support

           0       0.65      0.92      0.76        12
           1       0.88      0.54      0.67        13

    accuracy                           0.72        25
   macro avg       0.76      0.73      0.71        25
weighted avg       0.77      0.72      0.71        25



In [74]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from catboost import CatBoostRegressor
from scipy.stats import wasserstein_distance, mode
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
)
# ---------------------------------------- #
# 📌 증강 함수
def targeted_augment(X_embed, X_cat, X_news, y, ref_news_values, n_aug=3, noise_level=0.01):
    Xs, ys = [], []
    
    for i in range(len(X_embed)):
        for _ in range(n_aug):
            noise = np.random.normal(0, noise_level, size=X_embed.shape[1])
            embed_aug = X_embed[i] + noise
            
            # 🎯 기사수 증강: 테스트 분포에서 직접 샘플링
            news_aug = np.random.choice(ref_news_values)
            
            x_combined = np.concatenate([embed_aug, X_cat[i], [news_aug]])
            Xs.append(x_combined)
            ys.append(y[i])
    return np.array(Xs), np.array(ys)
                                  
def interpolate_augment(X_embed, X_cat, X_news, y, n_aug=3, news_reference=None):
    """
    - 임베딩, 카테고리, 타겟은 선형 보간
    - 기사수(news)는 전체 분포에서 무작위 샘플링하여 분포 왜곡 방지
    """
    Xs, ys = [], []
    n = len(X_embed)

    if news_reference is None:
        news_reference = X_news.flatten()

    for _ in range(n_aug * n):
        # 랜덤하게 두 샘플 선택
        i, j = np.random.randint(0, n, size=2)

        alpha = np.random.rand()  # 0~1 사이 가중치
        embed_mix = (1 - alpha) * X_embed[i] + alpha * X_embed[j]
        cat_mix = (1 - alpha) * X_cat[i] + alpha * X_cat[j]
        news_mix = np.random.choice(news_reference)  # 전체 분포에서 기사수 추출
        y_mix = (1 - alpha) * y[i] + alpha * y[j]

        x_combined = np.concatenate([embed_mix, cat_mix, [news_mix]])
        Xs.append(x_combined)
        ys.append(y_mix)

    return np.array(Xs), np.array(ys)



# ---------------------------------------- #
# 📊 분포 비교 시각화
def compare_three_distributions(X_train_real, X_train_augmented, X_test, y_train_real, y_train_augmented, y_test):
    print("\n📊 분포 비교 (원본 vs 증강 vs 테스트)")

    plt.figure(figsize=(14,5))

    # 1. 기사수 분포
    plt.subplot(1, 2, 1)
    sns.kdeplot(X_train_real[:, -1], label='Train (원본)', fill=True)
    sns.kdeplot(X_train_augmented[:, -1], label='Train (증강)', fill=True)
    sns.kdeplot(X_test[:, -1], label='Test', fill=True)
    plt.title("📰 기사수 분포")
    plt.legend()

    # 2. Y (상관계수) 분포
    plt.subplot(1, 2, 2)
    sns.kdeplot(y_train_real, label='Train Y (원본)', fill=True)
    sns.kdeplot(y_train_augmented, label='Train Y (증강)', fill=True)
    sns.kdeplot(y_test, label='Test Y', fill=True)
    plt.title("📊 상관계수 분포")
    plt.legend()

    plt.tight_layout()
    plt.show()

    # 정량적 Wasserstein 거리도 출력
    print(f"📏 기사수 거리 (원본 vs 테스트):     {wasserstein_distance(X_train_real[:, -1], X_test[:, -1]):.4f}")
    print(f"📏 기사수 거리 (증강 포함 vs 테스트): {wasserstein_distance(X_train_augmented[:, -1], X_test[:, -1]):.4f}")
    print(f"📏 Y 거리 (원본 vs 테스트):         {wasserstein_distance(y_train_real, y_test):.4f}")
    print(f"📏 Y 거리 (증강 포함 vs 테스트):   {wasserstein_distance(y_train_augmented, y_test):.4f}")

# ---------------------------------------- #
# 🚀 데이터 로드
print("🚀 데이터 로드 중...")
df = pd.read_excel(r"C:\Users\user\Desktop\파프\기사_전체_통합본 (상관계수).xlsx")
df["기사수"] = pd.to_numeric(df["기사수"], errors="coerce").fillna(0).astype(int)
df = df[df["설명력"].isin([0, 1])].copy()
df = df.dropna(subset=["특성추출(가중치순 상위 50개)"])
df.reset_index(drop=True, inplace=True)

# 📦 임베딩 로드
print("📦 임베딩 로드 중...")
with open(r"C:\Users\user\Desktop\파프\X_embed.pkl", "rb") as f:
    all_X_embed = pickle.load(f)
X_embed_all = all_X_embed[df.index]
assert X_embed_all.shape[0] == len(df)

# 📂 그룹 대표값 구성
category_cols = ['통합 분류1', '통합 분류2', '통합 분류3',
                 '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '감성']
grouped_rows = []
print("🔄 그룹별 대표값 생성 중...")

for (desc, corr), group in df.groupby(['설명력', '상관계수']):
    idxs = group.index.tolist()
    news_mean = group["기사수"].mean()
    mode_row = group[category_cols].mode().iloc[0]
    emb_mean = X_embed_all[idxs].mean(axis=0)

    row = {
        "설명력": desc,
        "상관계수": corr,
        "기사수": news_mean
    }
    for col in category_cols:
        row[col] = mode_row[col]
    grouped_rows.append((row, emb_mean))

df_new = pd.DataFrame([r for r, _ in grouped_rows])
X_embed = np.array([e for _, e in grouped_rows])

# 🔢 범주형 인코딩
onehot = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_cat = onehot.fit_transform(df_new[category_cols])
X_news = df_new[["기사수"]].values
y_all = df_new["상관계수"].astype(float).values

# 🔀 데이터 분할
print("✂️ 데이터 분리 중...")
idx_all = np.arange(len(y_all))
train_idx, test_idx = train_test_split(idx_all, test_size=0.3, random_state=42)

# Split by index
X_embed_train, X_embed_test = X_embed[train_idx], X_embed[test_idx]
X_cat_train, X_cat_test = X_cat[train_idx], X_cat[test_idx]
X_news_train, X_news_test = X_news[train_idx], X_news[test_idx]
y_train, y_test = y_all[train_idx], y_all[test_idx]

# 합치기
X_train_real = np.concatenate([X_embed_train, X_cat_train, X_news_train], axis=1)
X_test = np.concatenate([X_embed_test, X_cat_test, X_news_test], axis=1)

# 🧬 증강
X_aug, y_aug = interpolate_augment(
    X_embed_train, X_cat_train, X_news_train, y_train, n_aug=5
)


X_train_final = np.vstack([X_train_real, X_aug])
y_train_final = np.concatenate([y_train, y_aug])
print(f"🧬 증강 완료 ➤ 학습: {X_train_final.shape}, 검증: {X_test.shape}")

# 비교용
X_train_augmented = X_train_final  # (원본 + 증강)
y_train_augmented = y_train_final

# 그래프 + 거리 비교
compare_three_distributions(
    X_train_real=X_train_real,
    X_train_augmented=X_train_augmented,
    X_test=X_test,
    y_train_real=y_train,
    y_train_augmented=y_train_augmented,
    y_test=y_test
)


# 🎯 CatBoost 학습
print("🎯 CatBoostRegressor 학습 중...")
reg = CatBoostRegressor(
    iterations=500,
    learning_rate=0.05,
    depth=8,
    random_seed=42,
    od_type='Iter',
    od_wait=50,
    verbose=100
)
reg.fit(X_train_final, y_train_final)
print("✅ 학습 완료!")

# 📈 평가
print("📊 회귀 성능 평가:")
y_train_pred = reg.predict(X_train_real)
y_test_pred = reg.predict(X_test)

print("🧪 [Train Set]")
print(f"  🔹 MSE: {mean_squared_error(y_train, y_train_pred):.4f}")
print(f"  🔹 MAE: {mean_absolute_error(y_train, y_train_pred):.4f}")
print(f"  🔹 R²:  {r2_score(y_train, y_train_pred):.4f}")

print("🧪 [Test Set]")
print(f"  🔹 MSE: {mean_squared_error(y_test, y_test_pred):.4f}")
print(f"  🔹 MAE: {mean_absolute_error(y_test, y_test_pred):.4f}")
print(f"  🔹 R²:  {r2_score(y_test, y_test_pred):.4f}")


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 120)

In [59]:
# ----------------------------- #
# 12. 학습셋 및 테스트셋 성능 평가
print("\n📊 회귀 성능 평가:")

# 🔹 학습셋
y_train_pred = reg.predict(X_train_real)
print("🧪 [Train Set]")
print(f"  🔹 MSE: {mean_squared_error(y_train_real, y_train_pred):.4f}")
print(f"  🔹 MAE: {mean_absolute_error(y_train_real, y_train_pred):.4f}")
print(f"  🔹 R²:  {r2_score(y_train_real, y_train_pred):.4f}")

# 🔹 테스트셋
y_test_pred = reg.predict(X_test)
print("🧪 [Test Set]")
print(f"  🔹 MSE: {mean_squared_error(y_test, y_test_pred):.4f}")
print(f"  🔹 MAE: {mean_absolute_error(y_test, y_test_pred):.4f}")
print(f"  🔹 R²:  {r2_score(y_test, y_test_pred):.4f}")



📊 회귀 성능 평가:
🧪 [Train Set]
  🔹 MSE: 0.0000
  🔹 MAE: 0.0004
  🔹 R²:  1.0000
🧪 [Test Set]
  🔹 MSE: 0.0490
  🔹 MAE: 0.1468
  🔹 R²:  -0.1581


In [14]:
!pip install catboost optuna

     -------------------------------------- 386.6/386.6 kB 2.4 MB/s eta 0:00:00
     -------------------------------------- 242.5/242.5 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 45.8/45.8 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 78.5/78.5 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.10.0
    Uninstalling typing_extensions-4.10.0:
      Successfully uninstalled typing_extensions-4.10.0
